### In this notebook, we'are going to cover some of the most fundamental concepts of tensors using TensorFlow

#### More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercise to try for yourself

### Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.16.2


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

2025-03-24 12:21:16.743596: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-03-24 12:21:16.743770: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-24 12:21:16.743780: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-24 12:21:16.743995: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-24 12:21:16.744011: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check the number of dimensions of a tensor (`ndim` stands for number of dimensions)
scalar.ndim

0

In [5]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [6]:
print(vector)

tf.Tensor([10 10], shape=(2,), dtype=int32)


In [7]:
# Check out the dimension of our vector
vector.ndim

1

In [8]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7], [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [9]:
matrix.ndim

2

In [10]:
# Create another matrix
another_matrix = tf.constant([[1., 2., 3.],
    [4., 5., 6.],
    [7., 8., 9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 3), dtype=float16, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float16)>

In [11]:
# What's the ndim of the another_matrix
another_matrix.ndim

2

In [12]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
    [4, 5, 6]],
    [[7, 8, 9],
    [10, 11, 12]],
    [[13, 14, 15],
    [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [13]:
tensor.ndim

3

### What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

---
### Creating tensors with `tf.Variable`

In [14]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [15]:
# Create the same tensor with tf.Variable() as above

changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor


(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [16]:
# Let's try change one of the elements in our changeable tensor 

changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

This error happens because changeable_tensor is likely a TensorFlow tf.Variable, which does not support item assignment using standard Python syntax like changeable_tensor[0] = 7.

✅ Aaron's Notes - Why it happens:
tf.Variable is a mutable container for TensorFlow tensors, but you can't modify it in-place with normal indexing. Instead, TensorFlow provides specific methods to change its contents, such as `assign`, `assign_sub`, `assign_add`

In [17]:
# How about we try .assign()

changeable_tensor[0].assign(99)
changeable_tensor


<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([99,  7], dtype=int32)>

In [18]:
# Let's try change our unchangeable tensor

unchangeable_tensor[0] = 7
unchangeable_tensor

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [19]:
# Let's try `assign` to unchangeable constant

unchangeable_tensor[0].assign(99)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

#### 🔑 Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

#### Random tensors are tensors of some abitrary size which contain random numbers

In [23]:
# Create two random (but the same) tensors

random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.75658023, -0.06854693],
        [ 0.07595028, -1.2573844 ],
        [-0.23193759, -1.8107857 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.75658023, -0.06854693],
        [ 0.07595028, -1.2573844 ],
        [-0.23193759, -1.8107857 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [26]:
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

### Shuffle the order of elements in a tensor

In [33]:
# Shuffle a tensor (valueable for when you want to shuffle your data so the inherent order doesn't effect learning)

not_shuffled = tf.constant([[10, 7],
                           [3, 4],
                           [2, 5]])
                        
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [38]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [39]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### 🛠️ **Exercise:** 
1. Read through TensorFlow documentation on [random seed generation](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)
2. And, practice writing 5 random tensors and shufffle them.

In [ ]:
# Ex1: random tensors and shuffled (this is eager mode)
tf.random.set_seed(42)

print("before shuffling:", 30*"-")
r1 = tf.random.uniform([3, 2])
r2 = tf.random.uniform([3, 2])
print(r1, r2)

print("\nafter shuffling:", 30*"-")
print(tf.random.shuffle(r1))
print(tf.random.shuffle(r2))

before shuffling: ------------------------------
tf.Tensor(
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]
 [0.03366041 0.68467236]], shape=(3, 2), dtype=float32) tf.Tensor(
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]
 [0.73115396 0.89256823]], shape=(3, 2), dtype=float32)

after shuffling: ------------------------------
tf.Tensor(
[[0.03366041 0.68467236]
 [0.3528825  0.46448255]
 [0.6645621  0.44100678]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.73115396 0.89256823]
 [0.9309944  0.252187  ]
 [0.68789124 0.48447883]], shape=(3, 2), dtype=float32)


In [ ]:
# Ex2 - shuffle with decorators! (This is graph mode)

tf.random.set_seed(1234)

@tf.function
def shuffle_test():
    print("before shuffling:", 30*"-")
    r1 = tf.random.uniform([3, 2])
    r2 = tf.random.uniform([3, 2])
    print(r1, r2)
    print("\nafter shuffling:", 30*"-")
    print(tf.random.shuffle(r1))
    print(tf.random.shuffle(r2))

shuffle_test()

before shuffling: ------------------------------
Tensor("random_uniform/RandomUniform:0", shape=(3, 2), dtype=float32) Tensor("random_uniform_1/RandomUniform:0", shape=(3, 2), dtype=float32)

after shuffling: ------------------------------
Tensor("RandomShuffle:0", shape=(3, 2), dtype=float32)
Tensor("RandomShuffle_1:0", shape=(3, 2), dtype=float32)


2025-03-24 15:12:14.985377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [56]:
# Ex3 - Eager Mode Example:

import tensorflow as tf

# Set seed for reproducibility
tf.random.set_seed(42)

# Create 5 random tensors and put them into a list
tensors = [tf.random.uniform([2, 2]) for _ in range(3)]

print("Before shuffling:", 20*"=")
for i, t in enumerate(tensors):
    print(f"Tensor {i}: \n{t} \n")
    
# Stack them into a single tensor so we can shuffle them
stacked = tf.stack(tensors)  # shape = (5, 2, 2)
shuffled = tf.random.shuffle(stacked)

print("After shuffling:", 20*"=")
for i, t in enumerate(shuffled):
    print(f"Tensor {i}: \n{t} \n")

Before shuffling: ====================
Tensor 0: 
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.7413678  0.62854624]
 [0.01738465 0.3431449 ]] 

After shuffling: ====================
Tensor 0: 
[[0.7413678  0.62854624]
 [0.01738465 0.3431449 ]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]] 



### The above exercise, the **shuffle only happens across the first dimension,**, i.e., **between Tensor 0, 1, and 2**  -- not insdie each individual 2x2 tensor.

Let's say we have:
```python
tensors = [A, B, C]  # where each is a 2×2 tensor
stacked = tf.stack(tensors)  # shape = [3, 2, 2]
```

This give us a 3D tensor like:
```lua
[
  [[a1, a2],
   [a3, a4]],   # Tensor 0

  [[b1, b2],
   [b3, b4]],   # Tensor 1

  [[c1, c2],
   [c3, c4]]    # Tensor 2
]
```
When we call:
`tf.random.shuffle(stacked)`

We only are shuffling **the outer dimension** (axis 0), i.e., we're just changing the order of Tensor 0, Tensor 1, Tensor 2. as below:

```lua
→
[
  [[c1, c2],
   [c3, c4]],   # Was Tensor 2

  [[a1, a2],
   [a3, a4]],   # Was Tensor 0

  [[b1, b2],
   [b3, b4]]    # Was Tensor 1
]
```

### ✅ **What if we want to shuffle inside** each tensor?

In [59]:
# shuffle inside

print("Before shuffling INSIDE TENSOR:", 20*"=")
for i, t in enumerate(tensors):
    print(f"Tensor {i}: \n{t} \n")

shuffled_inside = [tf.random.shuffle(t) for t in tensors]

print("After shuffling INSDIE TENSOR:", 20*"=")
for i, t in enumerate(shuffled_inside):
    print(f"Tensor {i}: \n{t} \n")

Before shuffling INSIDE TENSOR: ====================
Tensor 0: 
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.7413678  0.62854624]
 [0.01738465 0.3431449 ]] 

After shuffling INSDIE TENSOR: ====================
Tensor 0: 
[[0.3528825  0.46448255]
 [0.6645621  0.44100678]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.01738465 0.3431449 ]
 [0.7413678  0.62854624]] 



In [60]:
tf.random.shuffle(not_shuffled, seed=42)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [61]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>

### if set global seed
#### If both the global and the operation seed are set: 
> - Both seeds are used in conjunction to determine the random sequence

### The above setting - **Both global and op-level seed are set** has the following behavior:

> #### - this is the `most stable` and `deterministi` setup
> #### - Combines both seeds deterministic to generate the result
> #### - We'll get `the same output` every time, even accross many random ops, and more robust to TensorFlow version changes!

In [63]:
tf.random.set_seed(42)  # global level random seed
tf.random.shuffle(not_shuffled, seed=42)  # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [64]:
tf.random.set_seed(42)  # global level random seed
tf.random.shuffle(not_shuffled, seed=42)  # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [65]:
tf.random.set_seed(42)  # global level random seed
tf.random.shuffle(not_shuffled, seed=42)  # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### ⬆️ same result each time

#### It looks like if we want our shuffled tensors to be in the same order, we've got6 to use the global level random seed as the operation level random seed:

> **Rule 4**: if both the global and the op-seed are set: Both seeds are used in conjunmtion to determine the random sequence.

### Other ways to make tensors

In [67]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [68]:
# Create a tensor of all zeros
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn Numpy arrays into tensors

#### The mian difference between Numpy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [70]:
# We can also turn NumPy arrays into tensors

import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32)  # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # cpaital for matrix or tensor
# y = tf.constant(vector)  # non-cap0ital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [71]:
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

### 🔍 So What's the Difference?

| Feature                     | `tf.constant()`                 | `tf.convert_to_tensor()`        |
|----------------------------|----------------------------------|----------------------------------|
| Creates a constant tensor  | ✅ Yes                           | ✅ Yes                            |
| Accepts NumPy, list, etc.  | ✅ Yes                           | ✅ Yes                            |
| Optimized for constants    | ✅                                | ❌ (more general)                |
| Used internally in APIs    | ❌                                | ✅                                |
| Can handle already-tensors | ❌ Will re-wrap it               | ✅ Returns as-is if already tensor |

---

## ✅ TL;DR – When to use which?

| Use Case                              | Recommended |
|---------------------------------------|-------------|
| You want a fixed tensor (won’t change) | `tf.constant()` |
| Writing flexible, general code         | `tf.convert_to_tensor()` |

In [74]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A, shape=(4, 6))
C = tf.constant(numpy_A)
A, B, C

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(4, 6), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

--- 
### **Getting information from tensors**

#### When dealing with tensors we probably want to be aware of the following attributes:

| **Attribute** | **Meaning** | **Code** |
|---|---|---|
| **Shape** |  The length (number of elements) of each of the dimensions of a tensor. | `tensor.shape` |
| **Rank** | The number of tensor dimensions. A scalar has rank 0, a vector has rank 1, a matrix is rank 2, a tensor has rank n. | `tensor.ndim` |
| **Axis or dimension** | A particular dimension of a tensor. | `tensor[0], tensor[:, 1]...` |
| **Size** | The total number of items in the tensor. | `tf.size(tensor)` |

In [76]:
# Create a rank 4 tensor (4 dimensions)\
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [77]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [84]:
print(
    rank_4_tensor.shape, '\n',
    rank_4_tensor.ndim, '\n',
    tf.size(rank_4_tensor))

(2, 3, 4, 5) 
 4 
 tf.Tensor(120, shape=(), dtype=int32)


In [ ]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of a tensor: ", rank_4_tensor.shape)
# print("Elements along the 0 axis: ", rank_4_tensor[0].shape)  # wrong

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of a tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  (3, 4, 5)


In [87]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of a tensor: ", rank_4_tensor.shape)
# print("Elements along the 0 axis: ", rank_4_tensor[0].shape)  # wrong
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of a tensor:  (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)


In [90]:
# Get various attributes of our tensor
# Notice the final statement converted to NumPy

print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of a tensor: ", rank_4_tensor.shape)
# print("Elements along the 0 axis: ", rank_4_tensor[0].shape)  # wrong
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of a tensor:  (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


---
### Indexing tensors

#### Tensors can be indexed just like Python lists.

In [92]:
# aaron's reshaping the original matrix

rank_4_tensor = tf.reshape(tf.range(120), shape=(2, 3, 4, 5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=int32, numpy=
array([[[[  0,   1,   2,   3,   4],
         [  5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14],
         [ 15,  16,  17,  18,  19]],

        [[ 20,  21,  22,  23,  24],
         [ 25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39]],

        [[ 40,  41,  42,  43,  44],
         [ 45,  46,  47,  48,  49],
         [ 50,  51,  52,  53,  54],
         [ 55,  56,  57,  58,  59]]],


       [[[ 60,  61,  62,  63,  64],
         [ 65,  66,  67,  68,  69],
         [ 70,  71,  72,  73,  74],
         [ 75,  76,  77,  78,  79]],

        [[ 80,  81,  82,  83,  84],
         [ 85,  86,  87,  88,  89],
         [ 90,  91,  92,  93,  94],
         [ 95,  96,  97,  98,  99]],

        [[100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109],
         [110, 111, 112, 113, 114],
         [115, 116, 117, 118, 119]]]], dtype=int32)>

In [93]:
# Get the first 2 elements of each dimensions
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int32, numpy=
array([[[[ 0,  1],
         [ 5,  6]],

        [[20, 21],
         [25, 26]]],


       [[[60, 61],
         [65, 66]],

        [[80, 81],
         [85, 86]]]], dtype=int32)>

In [95]:
# Get the first element from each dimension from each index
#   excpet for the final one

print(rank_4_tensor[:1, :1, :1])

tf.Tensor([[[[0 1 2 3 4]]]], shape=(1, 1, 1, 5), dtype=int32)


In [96]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [97]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=int32, numpy=
array([[[[ 0],
         [ 5],
         [10],
         [15]]]], dtype=int32)>

In [98]:
rank_4_tensor[:1, :, :1, :1]

<tf.Tensor: shape=(1, 3, 1, 1), dtype=int32, numpy=
array([[[[ 0]],

        [[20]],

        [[40]]]], dtype=int32)>

In [99]:
rank_4_tensor[:, :1, :1, :1]

<tf.Tensor: shape=(2, 1, 1, 1), dtype=int32, numpy=
array([[[[ 0]]],


       [[[60]]]], dtype=int32)>

In [101]:
# Create a rank 2 tensor (2 dimensions)

rank_2_tensor = tf.reshape(tf.range(24), shape=(4, 6))
rank_2_tensor

<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]], dtype=int32)>

In [102]:
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([4, 6]), 2)

In [ ]:
# Get the last item of each row of our rank 2 tensor

rank_2_tensor[-1, -1]  # wrong answer

<tf.Tensor: shape=(), dtype=int32, numpy=23>

In [ ]:
# Get the last item of each row of our rank 2 tensor

rank_2_tensor[-1:, -1:]  # wrong answer

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[23]], dtype=int32)>

In [ ]:
# Get the last item of each row of our rank 2 tensor
 
rank_2_tensor[:, -1]  

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 5, 11, 17, 23], dtype=int32)>

In [107]:
rank_2_tensor(axis=1)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable

In [113]:
# Using `tf.gather` with `axis=1`, but wrong because tensor does not support -1 index
rank_2_tensor = tf.reshape(tf.range(24), shape=(4, 6))
print(rank_2_tensor)

last_items_of_each_row = tf.gather(rank_2_tensor, indices=[-1], axis=1)
print(last_items_of_each_row)
tf.squeeze(last_items_of_each_row)

tf.Tensor(
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]], shape=(4, 6), dtype=int32)
tf.Tensor(
[[0]
 [0]
 [0]
 [0]], shape=(4, 1), dtype=int32)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>

In [114]:
# You'll need to calculate the last index, but not using -1

# Using `tf.gather` with `axis=1`, but wrong because tensor does not support -1 index
rank_2_tensor = tf.reshape(tf.range(24), shape=(4, 6))
print(rank_2_tensor)

last_items_of_each_row = tf.gather(rank_2_tensor, indices=[6-1], axis=1)
print(last_items_of_each_row)
tf.squeeze(last_items_of_each_row)

tf.Tensor(
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]], shape=(4, 6), dtype=int32)
tf.Tensor(
[[ 5]
 [11]
 [17]
 [23]], shape=(4, 1), dtype=int32)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 5, 11, 17, 23], dtype=int32)>

**TL;DR** for `tf.gather()` and `tf.squeeze()`:

---

## 🔹 `tf.gather()`

| Purpose                    | Grabs slices along an axis (like selecting rows or columns)        |
|----------------------------|---------------------------------------------------------------------|
| Common use                 | Select specific rows/columns (e.g., `axis=0` for rows, `axis=1` for columns) |
| Negative indices support?  | ❌ No — unlike NumPy, `-1` does **not** mean "last"                 |
| Fix for last index         | Use `tf.shape(tensor)[axis] - 1` instead of `-1`                   |
| Example                    | `tf.gather(tensor, indices=[2], axis=1)` → gets column 2 from all rows |

---

## 🔹 `tf.squeeze()`

| Purpose                    | Removes dimensions with size 1 (e.g., shape `[4, 1]` → `[4]`)        |
|----------------------------|-----------------------------------------------------------------------|
| When to use it             | After slicing/gathering when you want to flatten out unnecessary dims |
| Example                    | `tf.squeeze(tf.constant([[1], [2], [3]]))` → `[1, 2, 3]`             |
| Optional axis              | You can specify `axis` if you only want to squeeze specific dimensions |

---

### ✅ Example combo:

```python
last_col_index = tf.shape(tensor)[1] - 1
last_column = tf.gather(tensor, indices=[last_col_index], axis=1)
last_column = tf.squeeze(last_column)
```

---


In [115]:
# Add in extra dimension to our rank 2 tensor

rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(4, 6, 1), dtype=int32, numpy=
array([[[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5]],

       [[ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11]],

       [[12],
        [13],
        [14],
        [15],
        [16],
        [17]],

       [[18],
        [19],
        [20],
        [21],
        [22],
        [23]]], dtype=int32)>

**TL;DR for `[..., tf.newaxis]`** 👇

---

## 🔹 `[..., tf.newaxis]`

| Purpose                          | Adds a **new dimension** (axis of size 1) at the end of the tensor |
|----------------------------------|---------------------------------------------------------------------|
| Equivalent to                    | `tf.expand_dims(tensor, axis=-1)`                                  |
| Useful for                       | - Adding channel dimensions (e.g., grayscale images)                |
|                                  | - Preparing shape for broadcasting or model input                   |
| Shape change                     | From `[a, b]` → `[a, b, 1]`                                         |
| Can also insert in other axes   | Yes! Use slicing like `tensor[:, tf.newaxis, :]` to insert at axis 1 |

---

## ✅ Example:

```python
x = tf.constant([1, 2, 3])        # shape: (3,)
x_expanded = x[..., tf.newaxis]   # shape: (3, 1)
```

---

### 🧠 Bonus: Why use `...`?

- `...` is a shortcut for "all previous dimensions"
- So `[..., tf.newaxis]` means: "add a new axis at the very end"

---


In [116]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand final axis


<tf.Tensor: shape=(4, 6, 1), dtype=int32, numpy=
array([[[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5]],

       [[ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11]],

       [[12],
        [13],
        [14],
        [15],
        [16],
        [17]],

       [[18],
        [19],
        [20],
        [21],
        [22],
        [23]]], dtype=int32)>

In [ ]:
tf.expand_dims(rank_2_tensor, axis=0)  # expand the 0-axis

<tf.Tensor: shape=(1, 4, 6), dtype=int32, numpy=
array([[[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]], dtype=int32)>